

| EDA dataframe      | Description |due|
| ----------- | ----------- |----------- |
| grape_df  | 캠벨얼리 가격/거래량 + 소비자 물가 지수 + 자동차용경유 + 날씨(4개-평균기온, 평균온량, 일강수량, 일교차) + 수출입(4개) + 재배면적 + 샤인머스켓 거래량 + 실업률_경제활동인구(2개-경제활동인구, 실업률) + 최저시급    |20190101 ~ 20201104|
| shinemuscat_df   |샤인마스캇  가격/거래량 + 소비자 물가 지수 + 자동차용경유 + 날씨(4개-평균기온, 평균온량, 일강수량, 일교차) + 수출입(4개) + 재배면적 + 블루베리 거래량 + 실업률_경제활동인구(2개-경제활동인구, 실업률) + 최저시급        |20190101 ~ 20201104|
| df      | grape_train + shinemuscat_train       | 20190101 ~ 2020928|


**train accuracy / test accuracy 의 차이 비교 overfitting = 20% 이상 (10~20%)**

https://dacon.io/competitions/official/235801/codeshare/4063?page=1&dtype=recent

[lgbm 튜닝](https://dacon.io/competitions/official/235698/talkboard/404322)

In [1]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic'

from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose, STL
import urllib.request
import requests
import json
import datetime
import os
from datetime import datetime
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

/var/folders/lm/z4n_2b8s6rj3_vv57sg789kc0000gn/T/ipykernel_12602/2031877544.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# iMAC

raw_path = '/Users/cge/Documents/yeardream/project/kurly/products_price_predict/data/raw/'
processed_path = '/Users/cge/Documents/yeardream/project/kurly/products_price_predict/data/processed/'

In [1]:
# Macbook
    
# raw_path = '/Users/goeun/Documents/YearDream/ProJect/kurly_project/products_price_predict/data/raw/'
# processed_path = '/Users/goeun/Documents/YearDream/ProJect/kurly_project/products_price_predict/data/processed/'

In [4]:
# tomato = pd.read_csv(processed_path+'tomato.csv')
grape = pd.read_csv(processed_path+'캠벨얼리_전처리후.csv')
shinemuscat = pd.read_csv(processed_path+'샤인머스켓_전처리후.csv')

train = pd.read_csv(raw_path+'train.csv')
test = pd.read_csv(raw_path+'test.csv')
private = pd.read_csv(raw_path+'private_data.csv')
submission = pd.read_csv(raw_path+'sample_submission.csv')

print(grape.shape)
print(shinemuscat.shape)
print(private.shape)
print(submission.shape)

(1770, 9)
(1770, 9)
(291, 44)
(228, 22)


In [6]:
grape_external = pd.read_csv(processed_path+'grape_실업률추가.csv')
shinemuscat_external = pd.read_csv(processed_path+'shinemuscat_실업률추가.csv')
grape_external.columns, shinemuscat_external.columns

(Index(['date', '요일', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)', '소비자물가지수', '자동차용경유',
        '재배지역_평균기온', '재배지역_평균운량', '재배지역_일강수량', '재배지역_일교차', '샤인마스캇_거래량(kg)',
        '수출(중량)', '수출(금액)', '수입(중량)', '수입(금액)', '재배면적(ha)', '최저시급', '경제활동인구',
        '실업률 (%)'],
       dtype='object'),
 Index(['date', '요일', '샤인마스캇_거래량(kg)', '샤인마스캇_가격(원/kg)', '소비자물가지수', '자동차용경유',
        '재배지역_평균기온', '재배지역_평균운량', '재배지역_일강수량', '재배지역_일교차', '수출(중량)', '수출(금액)',
        '수입(중량)', '수입(금액)', '재배면적(ha)', '블루베리_거래물량(kg)', '최저시급', '경제활동인구',
        '실업률 (%)'],
       dtype='object'))

In [9]:
grape_scaler2['date'] = pd.to_datetime(grape_scaler2['date'])
grape_scaler2.set_index('date',drop=True,inplace=True)

shinemuscat_scaler2['date'] = pd.to_datetime(shinemuscat_scaler2['date'])
shinemuscat_scaler2.set_index('date',drop=True,inplace=True)

# LGBM(LightGBM)

## 캠벨얼리
성능을 알아보기 위한 모델 <br> 캠벨얼리 가격을 예측
[lightgbm](https://tpwkcorqhd.tistory.com/32)

In [7]:
private2 = private.copy()
private2.drop(columns=['요일','배추_거래량(kg)', '배추_가격(원/kg)', '무_거래량(kg)', '무_가격(원/kg)',
       '양파_거래량(kg)', '양파_가격(원/kg)', '건고추_거래량(kg)', '건고추_가격(원/kg)',
       '마늘_거래량(kg)', '마늘_가격(원/kg)', '대파_거래량(kg)', '대파_가격(원/kg)',
       '얼갈이배추_거래량(kg)', '얼갈이배추_가격(원/kg)', '양배추_거래량(kg)', '양배추_가격(원/kg)',
       '깻잎_거래량(kg)', '깻잎_가격(원/kg)', '시금치_거래량(kg)', '시금치_가격(원/kg)',
       '미나리_거래량(kg)', '미나리_가격(원/kg)', '당근_거래량(kg)', '당근_가격(원/kg)',
       '파프리카_거래량(kg)', '파프리카_가격(원/kg)', '새송이_거래량(kg)', '새송이_가격(원/kg)',
       '팽이버섯_거래량(kg)', '팽이버섯_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)',
       '청상추_거래량(kg)', '청상추_가격(원/kg)', '백다다기_거래량(kg)', '백다다기_가격(원/kg)',
       '애호박_거래량(kg)', '애호박_가격(원/kg)','샤인마스캇_거래량(kg)', '샤인마스캇_가격(원/kg)'], inplace=True)

In [8]:
private2['date'] = private2.date.astype(str).str.replace('-','')
private2['date'] = pd.to_datetime(private2.date, format='%Y%m%d')


t = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
private2['요일'] = 0
temp = []
for i in range(len(private2)):
      temp.append(t[int(private2['date'][i].weekday())])
private2['요일'] = temp

private2 = private2[['date', '요일', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)',]]

In [52]:
grape.columns

Index(['date', '요일', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)', '캠벨얼리_거래량의_1주전_평균',
       '캠벨얼리_가격의_1주전_평균', '캠벨얼리_거래량의_2주전_평균', '캠벨얼리_가격의_2주전_평균', '계절'],
      dtype='object')

In [109]:
class Nong1:
    

  def __init__(self, df, test):
    self.df = df
    self.test = test
    self.df = pd.concat([self.df, self.test], axis=0)
    self.df = self.df[1:].replace(0, np.NaN)
#     self.df = self.df.interpolate().fillna(0)
    self.df.date = pd.to_datetime(self.df.date)
    self.df = pd.concat([self.df, pd.get_dummies(self.df['요일'])], axis=1)
    self.feature = self.df.columns[2:]
    self.df = self.df.reset_index(drop=True)

  def set_feature(self,name):
    self.name = name
    self.name1 = name+ "_가격(원/kg)"
    self.name2 = name+ "_거래량(kg)"
    self.name3 = name+ "_거래량의_1주전_평균"
    self.name4 = name+ '_가격의_1주전_평균'
    self.name5 = name+ '_거래량의_2주전_평균'
    self.name6 = name+ '_가격의_2주전_평균'
    self.name7 = '계절'
    self.feature = [self.name1, self.name2, self.name3, self.name4, self.name5, self.name6, self.name7,'금요일',   '목요일',   '수요일',   '월요일',   '일요일',   '토요일',   '화요일', 'resid']

  def set_target(self,week):
    if week == 1:
      self.df['target'] = self.df[self.name1].shift(-8)
    elif week == 2:
      self.df['target'] = self.df[self.name1].shift(-15)
    
    self.df['resid'] = 0
    stl = STL(self.df[['date', self.name1]].set_index('date'), period=12)
    res = stl.fit()
    self.df['resid'] = res.resid.values

  def set_model_lgb(self):
    import lightgbm as LGBMRegressor
    self.scaler = RobustScaler()
    self.df[self.feature] = self.scaler.fit_transform(self.df[self.feature])
    self.df_learn = self.df[self.df['target'].notnull()]
    self.X = self.df_learn[self.feature]
    self.y = self.df_learn['target']
    
    self.valid_split = 1770
    
    self.X_train = self.df_learn[self.feature][:self.valid_split].values
    self.y_train = self.df_learn['target'][:self.valid_split].values
    self.X_test = self.df_learn[self.feature][self.valid_split:].values
    self.y_test = self.df_learn['target'][self.valid_split:].values
    # self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, shuffle=False)

    self.params= {
                'metric':'mae',
                'seed':42     
                }  
#     self.num_round = 10
    
#     self.train_ds = lgb.Dataset(self.X_train,  label = self.y_train)
#     self.test_ds = lgb.Dataset(self.X_test,  label = self.y_test)

    self.model = LGBMRegressor(n_estimators=30, learning_rate=0.12)
    self.model.fit(X_train, y_train)
      
  def get_plot(self):  

    self.y_pred = self.model.predict(self.X)
    self.RMSE = mean_squared_error(self.y, self.y_pred) ** 0.5

    plt.figure(figsize=(20, 10), dpi=300)
    plt.title(self.name + ' 가격 예측 결과'+   '   RMSE : ' + str(self.RMSE)[:7])
    plt.ylabel(self.name + ' 가격')
    plt.plot(np.array(self.y.reshape(-1,1)), alpha = 0.9, label = 'Real')
    plt.plot(self.model.predict(self.X).reshape(-1,1), alpha = 0.6, linestyle = "--", label = 'Predict')
    plt.legend()
    plt.show()

  def get_price(self):
    self.price = self.model.predict(self.df[self.feature].iloc[len(self.df)-1].values)
    return self.price[0][0]

In [110]:
my_nong1 = Nong1(grape, private2)

my_nong1.set_feature('캠벨얼리')

my_nong1.set_target(1)

In [111]:
my_nong1.set_model_lgb()

TypeError: 'module' object is not callable

In [108]:
my_nong1.get_plot()

AttributeError: 'Nong1' object has no attribute 'model'

### 모델 실행


In [50]:
weeks = [1,2,4]
features = ['캠벨얼리',] # 새송이버섯

week1=[]
week2=[]

for week in weeks:
  print(week)
  for feature in features:
    my_nong1 = Nong1(grape, private2)
    my_nong1.set_feature(feature)
    my_nong1.set_target(week)
    print(my_nong1.set_target(week))
    my_nong1.set_model()
    if week == 1:
      week1.append(my_nong1.get_price())
    if week == 2:
      week2.append(my_nong1.get_price())
    print(feature)

1
None


AttributeError: 'Nong1' object has no attribute 'set_model'